In [1]:
import glob
import os
import argparse
import shutil
import tempfile
import time
from datetime import datetime
import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
from torch import nn
import torch.optim as optim
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from torchsummary import summary
import random
import numpy as np
import configparser
import gc
import math
#from utils.training_torch_utils import BoxCrop, Dulicated, train, validation, Data_progressing, plot_loss_metric
#import utils.config as config
#from efficientnet_3d.model_3d import EfficientNet3D
#from resnet_3d import resnet_3d
#from ModelsGenesis import unet3d
import sys
sys.path.append("/tf/jacky831006/classification_torch/NAS-Lung/") 
from models.cnn_res import *

from monai.config import print_config
from monai.data import CacheDataset, DataLoader, Dataset
from monai.metrics import compute_meandice
from monai.networks.layers import Norm
from monai.networks.nets import UNet, densenet ,SENet
from monai.utils import set_determinism
from monai.transforms import (
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadNiftid,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    ToTensord,
    Resized,
    RandAffined,
    Rand3DElasticd
)
print_config()

MONAI version: 0.4.0
Numpy version: 1.18.4
Pytorch version: 1.6.0+cu101
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0563a4467fa602feca92d91c7f47261868d171a1

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.1.0
scikit-image version: 0.17.2
Pillow version: 8.0.1
Tensorboard version: 2.2.1
gdown version: 3.12.2
TorchVision version: 0.7.0+cu101
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.54.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
class AngleLoss_predict(nn.Module):
    def __init__(self, gamma=0):
        super(AngleLoss_predict, self).__init__()
        self.gamma = gamma
        self.it = 1
        self.LambdaMin = 5.0
        self.LambdaMax = 1500.0
        self.lamb = 1500.0

    def forward(self, input, target):
        cos_theta, phi_theta = input
        target = target.view(-1, 1)  # size=(B,1)
        index = cos_theta.data * 0.0  # size=(B, Classnum)
        # index = index.scatter(1, target.data.view(-1, 1).long(), 1)
        #index = index.byte()
        index = index.bool()  
        index = Variable(index)
        # index = Variable(torch.randn(1,2)).byte()

        self.lamb = max(self.LambdaMin, self.LambdaMax / (1 + 0.1 target* self.it))
        output = cos_theta * 1.0  # size=(B,Classnum)
        output1 = output.clone()
        # output1[index1] = output[index] - cos_theta[index] * (1.0 + 0) / (1 + self.lamb)
        # output1[index1] = output[index] + phi_theta[index] * (1.0 + 0) / (1 + self.lamb)
        output[index] = output1[index]- cos_theta[index] * (1.0 + 0) / (1 + self.lamb)+ phi_theta[index] * (1.0 + 0) / (1 + self.lamb)
        return(output)

In [3]:
def test(model, testLoader, device):
    pre_first = True
    model.eval()
    with torch.no_grad():
        num_correct = 0.0
        metric_count = 0
        for testdata in testLoader:
            test_images, test_labels = testdata['image'].to(device), testdata['label'].to(device)
            output = model(test_images)
            if isinstance(output, tuple):
                output = AngleLoss_predict()(output,test_labels)
            # output 並非0-1之間 故進行轉換
            pre = nn.functional.softmax(output,dim=1).cpu().detach().numpy()
            # 比較output與label 對的話則返回 true 錯則 false
            value = torch.eq(output.argmax(dim=1), test_labels)
            metric_count += len(value)
            num_correct += value.sum().item()
            if pre_first:
                pre_first = None
                predict_values = pre
            else:
                predict_values = np.concatenate((predict_values,pre),axis=0)
        metric = num_correct / metric_count
        print("Test Accuracy: {}".format(num_correct / metric_count))
        return (predict_values)

In [4]:
net = ConvRes(32,[[64, 64, 64], [128, 128, 256], [256, 256, 256, 512]])
net = net.cuda()

In [5]:
device = torch.device("cuda",0)
inputs = torch.randn((8, 1, 32, 32, 32)).to(device)
labels = torch.randn((1)).to(device)
output = net(inputs)
print(output)

(tensor([[-1.2158, -0.1602],
        [-1.2669, -0.2861],
        [-1.0327, -0.4320],
        [-1.3313, -0.2058],
        [-1.2232, -0.3826],
        [-1.0999, -0.0213],
        [-1.4132, -0.1492],
        [-1.2426, -0.4877]], device='cuda:0', grad_fn=<MulBackward0>), tensor([[-81.3050, -80.8748],
        [-83.4876, -83.0482],
        [-75.8539, -75.5748],
        [-78.9563, -78.4282],
        [-81.8026, -81.4053],
        [-82.0370, -81.6822],
        [-84.9888, -84.4288],
        [-81.4677, -81.0817]], device='cuda:0', grad_fn=<MulBackward0>))


In [21]:
class GradCAM(object):
    """
    1: 网络不更新梯度,输入需要梯度更新
    2: 使用目标类别的得分做反向传播
    """

    def __init__(self, net, layer_name, device):
        self.net = net
        self.layer_name = layer_name
        self.feature = None
        self.gradient = None
        self.device = device
        self.net.eval()
        self.handlers = []
        # 將 feature與gradient 取出
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        self.feature = output
        #print("feature shape:{}".format(output.size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """
        :param input_grad: tuple,  input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grlabelsad[2]: bias
        :param output_grad:tuple,  长度为1
        :return:
        """
        self.gradient = output_grad[0]

    def _register_hook(self):
        for (name, module) in self.net.named_modules():
            if name == self.layer_name:
                #print("OK")
                self.handlers.append(module.register_forward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))
            #else:
                #print("Nothing to do")

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, index_sel=None):
        """
        :param inputs: {"image": [C,H,W], "height": height, "width": width}
        :param index_sel: 第几个边框
        :return:  a list of each batch heatmap
        """
        # 將 feature與gradient 取出
        #self._register_hook()
        # forward
        # with torch.no_grad():

        self.net.zero_grad()
        output = self.net(inputs) # [1,num_classes]
        heatmap_list = []
        print(f"output shape: {output[0].shape}")
        #print(output)
        for i in range(output[0].shape[0]):
            if  index_sel == None:
                index = np.argmax(output[0].data[i,:].cpu().data.numpy())
                print(f"predict:{index}")  
            else:
                index = index_sel
            # backward
            target = output[0][i][index]
            print(target)
            target.backward(retain_graph=True)
            print(self.gradient)
            gradient = self.gradient[i].cpu().detach().data.numpy()  # [C,H,W,D]
            weight = np.mean(gradient, axis=(1, 2, 3))  # [C]

            feature = self.feature[i].cpu().detach().data.numpy()  # [C,H,W,D]
            
            # np.newaxis 增加維度的方法
            cam = feature * weight[:, np.newaxis, np.newaxis, np.newaxis]  # [C,H,W,D] feature map 與 weight相乘
            cam = np.sum(cam, axis=0)  # [H,W,D] 
            cam = np.maximum(cam, 0)  # ReLU
            heatmap = (cam - cam.min()) / (cam.max() - cam.min())
            heatmap = resize(heatmap,inputs.shape[2:5])
            heatmap_list.append(heatmap)

        return heatmap_list

In [51]:
class GradCAM_2(object):
    """
    1: 网络不更新梯度,输入需要梯度更新
    2: 使用目标类别的得分做反向传播
    """

    def __init__(self, net, layer_name, device):
        self.net = net
        self.layer_name = layer_name
        self.feature = None
        self.gradient = None
        self.device = device
        self.net.eval()
        self.handlers = []
        # 將 feature與gradient 取出
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        self.feature = output
        print(self.feature)
        print("feature shape:{}".format(output.size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """
        :param input_grad: tuple,  input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grad[2]: bias
        :param output_grad:tuple,  长度为1
        :return:
        """
        self.gradient = output_grad[0]
        print(self.gradient)

    def _register_hook(self):
        for (name, module) in self.net.named_modules():
            if name == self.layer_name:
                #print("OK")
                self.handlers.append(module.register_forward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))
            #else:
                #print("Nothing to do")

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, labels, index_sel=None):
        """
        :param inputs: {"image": [C,H,W], "height": height, "width": width}
        :param index_sel: 第几个边框
        :return:  a list of each batch heatmap
        """
        # 將 feature與gradient 取出
        #self._register_hook()
        # forward
        # with torch.no_grad():

        self.net.zero_grad()
        output = self.net(inputs) # [1,num_classes]
        
        print(output[0].grad_fn)
        if isinstance(output, tuple):
            #output = AngleLoss_predict()(output,test_labels)
            output2 = output[0]
            print(output2)
            print(output2.grad_fn)
            output = Variable(output)
        heatmap_list = []
        #print(f"output shape: {output.shape}")
        
        for i in range(output.shape[0]):
            if  index_sel == None:
                index = np.argmax(output[i,:].cpu().data.numpy())
                print(f"predict:{index}")  
            else:
                index = index_sel
            # backward
            target = output[i][index]
            target.backward(retain_graph=True)

            gradient = self.gradient[i].cpu().detach().data.numpy()  # [C,H,W,D]
            weight = np.mean(gradient, axis=(1, 2, 3))  # [C]

            feature = self.feature[i].cpu().detach().data.numpy()  # [C,H,W,D]
            
            # np.newaxis 增加維度的方法
            cam = feature * weight[:, np.newaxis, np.newaxis, np.newaxis]  # [C,H,W,D] feature map 與 weight相乘
            cam = np.sum(cam, axis=0)  # [H,W,D] 
            cam = np.maximum(cam, 0)  # ReLU
            heatmap = (cam - cam.min()) / (cam.max() - cam.min())
            heatmap = resize(heatmap,inputs.shape[2:5])
            heatmap_list.append(heatmap)

        return heatmap_list

In [48]:
def get_last_conv_name(net):
    """
    获取网络的最后一个卷积层的名字
    :param net:
    :return:
    """
    #layer_name = None
    layer_name_list = []
    for name, m in net.named_modules():
        #print(name)
        #print(m)
        if isinstance(m, nn.Conv3d):
            layer_name_list.append(name)
            #layer_name = name
    return layer_name_list

In [49]:
layer_name = get_last_conv_name(net)[-1]
grad_cam = GradCAM(net, layer_name, device)
result_list = grad_cam(inputs)

feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
output shape: torch.Size([8, 2])
predict:1
tensor(-0.0530, device='cuda:0', grad_fn=<SelectBackward>)
tensor([[[[[-0.0001, -0.0001, -0.0001, -0.0001],
           [-0.0001, -0.0001, -0.0001, -0.0001],
           [-0.0001, -0.0001, -0.0001, -0.0001],
           [-0.0001, -0.0001, -0.0001, -0.0001]],

          [[-0.0002, -0.0001, -0.0001, -0.0001],
           [-0.0002, -0.0001, -0.0001, -0.0001],
           [-0.0002, -0.0002, -0.0002, -0.0001],
           [-0.0001, -0.0001, -0.0001, -0.0001]],

          [[-0.0002, -0.0001, -0.0001, -0.0001],
           [-0.0002, -0.0001, -0.

NameError: name 'resize' is not defined

In [52]:
layer_name = get_last_conv_name(net)[-1]
grad_cam = GradCAM_2(net, layer_name, device)
result_list = grad_cam(inputs)

feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
feature shape:torch.Size([8, 1, 4, 4, 4])
tensor([[[[[ 4.8192e-05, -4.0527e-06, -7.8900e-06, -5.8108e-05],
           [ 9.9255e-05,  6.8981e-05,  6.2890e-05, -3.6464e-05],
           [ 9.4997e-05,  5.9438e-05,  5.9109e-05, -3.9836e-05],
           [-1.9037e-06, -4.0569e-06, -6.6503e-06, -6.8057e-05]],

          [[ 2.0735e-05, -9.7266e-05, -9.7874e-05, -1.2328e-04],
           [ 8.2801e-05,  9.0794e-06,  8.1685e-06, -7.5284e-05],
           [ 8.2110e-05,  5.4205e-06,  7.9309e-06, -7.6713e-05],
           [ 9.2403e-06,  1.9504e-06,  1.4520e-07, -7.8511e-05]],

          [[ 2

TypeError: Variable data has to be a tensor, but got tuple

In [17]:
tar = Variable(torch.Tensor(1)).cuda()
x = torch.randn((1, 1, 32, 32, 32)).cuda()
out = net(x)
cre = AngleLoss()
loss = cre(out, tar)
print(loss)
loss.backward()

torch.Size([1, 512])
tensor(0.3317, device='cuda:0', grad_fn=<MeanBackward0>)


In [33]:
val_outputs=AngleLoss_test()(out,tar)
#val_outputs = loss_test(out,tar)
value = torch.eq(val_outputs.argmax(dim=1), tar)
print(value)

tensor([False], device='cuda:0')


In [5]:
net2 = ConvRes(32,[[64, 64, 64], [128, 128, 256], [256, 256, 256, 512]],normal=True)
net2 = net2.cuda()
inputs = torch.randn((1, 1, 32, 32, 32)).cuda()
labels = torch.randn((1)).cuda()
output = net2(inputs)
print(net2.config)
print(output)

[[64, 64, 64], [128, 128, 256], [256, 256, 256, 512]]
tensor([[ 1.1392, -0.5168]], device='cuda:0', grad_fn=<AddmmBackward>)


In [22]:
class AngleLoss_test(nn.Module):
    def __init__(self, gamma=0):
        super(AngleLoss_test, self).__init__()
        self.gamma = gamma
        self.it = 1
        self.LambdaMin = 5.0
        self.LambdaMax = 1500.0
        self.lamb = 1500.0

    def forward(self, input, target):
        cos_theta, phi_theta = input
        target = target.view(-1, 1)  # size=(B,1)
        index = cos_theta.data * 0.0  # size=(B, Classnum)
        # index = index.scatter(1, target.data.view(-1, 1).long(), 1)
        #index = index.byte()
        index = index.bool()  
        index = Variable(index)
        # index = Variable(torch.randn(1,2)).byte()

        self.lamb = max(self.LambdaMin, self.LambdaMax / (1 + 0.1 * self.it))
        output = cos_theta * 1.0  # size=(B,Classnum)
        output1 = output.clone()
        # output1[index1] = output[index] - cos_theta[index] * (1.0 + 0) / (1 + self.lamb)
        # output1[index1] = output[index] + phi_theta[index] * (1.0 + 0) / (1 + self.lamb)
        output[index] = output1[index]- cos_theta[index] * (1.0 + 0) / (1 + self.lamb)+ phi_theta[index] * (1.0 + 0) / (1 + self.lamb)
        return output
        

In [15]:
value = torch.eq(out.argmax(dim=1), val_labels)
metric_count += len(value)
num_correct += value.sum().item()
metric = num_correct / metric_count
config.metric_values.append(metric)

AttributeError: 'tuple' object has no attribute 'argmax'

In [4]:
net2 = ConvRes(64,[[64, 64, 64], [128, 128, 256], [256, 256, 256, 512]])
net2 = net2.cuda()

In [5]:
inputs = torch.randn((1, 1, 64, 64, 64)).cuda()
output = net2(inputs)

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([512, 2])
Parameter containing:
tensor([[-0.0089,  0.0317],
        [ 0.0261,  0.0550],
        [ 0.0073, -0.0149],
        ...,
        [ 0.0007, -0.0344],
        [-0.0294, -0.0512],
        [-0.0696, -0.0571]], device='cuda:0', requires_grad=True)
torch.Size([512, 2])


In [7]:
summary(net,(1,32,32,32))

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([512, 2])
Parameter containing:
tensor([[-0.0730,  0.0627],
        [-0.0516, -0.0192],
        [ 0.0367, -0.0305],
        ...,
        [ 0.0088,  0.0687],
        [ 0.0025, -0.0658],
        [ 0.0702, -0.0125]], device='cuda:0', requires_grad=True)
torch.Size([512, 2])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 4, 32, 32, 32]             112
       BatchNorm3d-2        [-1, 4, 32, 32, 32]               8
              ReLU-3        [-1, 4, 32, 32, 32]               0
            Conv3d-4        [-1, 4, 32, 32, 32]             436
       BatchNorm3d-5        [-1, 4, 32, 32, 32]               8
              ReLU-6        [-1, 4, 32, 32, 32]               0
         AvgPool3d-7           [-1, 4, 1, 1, 1]               0
         MaxPool3d-8           [-1, 4, 1, 1, 1]               0
            Linear-9              

In [6]:
summary(net2,(1,64,64,64))

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([512, 2])
Parameter containing:
tensor([[-0.0089,  0.0317],
        [ 0.0261,  0.0550],
        [ 0.0073, -0.0149],
        ...,
        [ 0.0007, -0.0344],
        [-0.0294, -0.0512],
        [-0.0696, -0.0571]], device='cuda:0', requires_grad=True)
torch.Size([512, 2])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 4, 64, 64, 64]             112
       BatchNorm3d-2        [-1, 4, 64, 64, 64]               8
              ReLU-3        [-1, 4, 64, 64, 64]               0
            Conv3d-4        [-1, 4, 64, 64, 64]             436
       BatchNorm3d-5        [-1, 4, 64, 64, 64]               8
              ReLU-6        [-1, 4, 64, 64, 64]               0
         AvgPool3d-7           [-1, 4, 1, 1, 1]               0
         MaxPool3d-8           [-1, 4, 1, 1, 1]               0
            Linear-9              

In [6]:
model = densenet.densenet121(spatial_dims=3, in_channels=1, out_channels=2).cuda()

In [7]:
summary(model,(1,128,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 64, 64, 64]          21,952
       BatchNorm3d-2       [-1, 64, 64, 64, 64]             128
              ReLU-3       [-1, 64, 64, 64, 64]               0
         MaxPool3d-4       [-1, 64, 32, 32, 32]               0
       BatchNorm3d-5       [-1, 64, 32, 32, 32]             128
              ReLU-6       [-1, 64, 32, 32, 32]               0
            Conv3d-7      [-1, 128, 32, 32, 32]           8,192
       BatchNorm3d-8      [-1, 128, 32, 32, 32]             256
              ReLU-9      [-1, 128, 32, 32, 32]               0
           Conv3d-10       [-1, 32, 32, 32, 32]         110,592
      _DenseLayer-11       [-1, 96, 32, 32, 32]               0
      BatchNorm3d-12       [-1, 96, 32, 32, 32]             192
             ReLU-13       [-1, 96, 32, 32, 32]               0
           Conv3d-14      [-1, 128, 32,

In [16]:
inputs = torch.randn((1, 1, 64, 64, 64)).cuda()
output2 = model(inputs)
print(output2)

tensor([[0.0809, 0.3521]], device='cuda:0', grad_fn=<AddmmBackward>)


In [53]:
import torchvision.models as models

In [ ]:
models.v